In [323]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [324]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgbm
import gc
import xgboost as xgb

In [325]:
items           = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')
sample_submission            = pd.read_csv('sample_submission.csv')

In [326]:
transactions = sales_train
transactions[['day','month', 'year']] = transactions['date'].str.split('.', expand=True).astype(int)


In [327]:
transactions = transactions.set_index('item_id').join(items.set_index('item_id'))
transactions.reset_index(inplace=True)
transactions = transactions[transactions['year'] != 2013]
transactions['y'] = transactions.groupby(['date_block_num', 'item_id', 'shop_id'])['item_cnt_day'].transform('sum').clip(0,20)


In [328]:
print(len(transactions))
transactions.head()

1668287


,item_id,date,date_block_num,shop_id,item_price,item_cnt_day,day,month,year,item_name,item_category_id,y
0,0,01.09.2014,20,54,58.0,1.0,1,9,2014,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,40,1.0
1,1,04.04.2014,15,55,4490.0,1.0,4,4,2014,"!ABBYY FineReader 12 Professional Edition Full [PC, Цифровая версия]",76,2.0
2,1,02.04.2014,15,55,4490.0,1.0,2,4,2014,"!ABBYY FineReader 12 Professional Edition Full [PC, Цифровая версия]",76,2.0
3,1,06.07.2014,18,55,4490.0,1.0,6,7,2014,"!ABBYY FineReader 12 Professional Edition Full [PC, Цифровая версия]",76,1.0
4,1,04.08.2014,19,55,4490.0,1.0,4,8,2014,"!ABBYY FineReader 12 Professional Edition Full [PC, Цифровая версия]",76,1.0


In [357]:
number_of_items = transactions['item_id'].nunique()
print("number_of_items:", number_of_items)
number_of_categories = transactions['item_category_id'].nunique()
print("number_of_categories:", number_of_categories)
number_of_shops = transactions['shop_id'].nunique()
print("number_of_shops:", number_of_shops)
number_of_days = 365 + 365 - 30 - 31
print("number_of_days:", number_of_days)
number_of_months = transactions['date_block_num'].nunique()
print("number_of_months:", number_of_months)
total_sales = transactions['item_cnt_day'].sum()
print("total_sales:", total_sales)

number_of_items: 17054
number_of_categories: 79
number_of_shops: 55
number_of_days: 669
number_of_months: 22
total_sales: 2085473.0


In [330]:
transactions['total_sales_units'] = transactions.groupby(['item_id'])['item_cnt_day'].transform(np.sum)

transactions['avg_sales_month'] = transactions.groupby(['item_id','date_block_num'])['item_cnt_day'].transform(np.sum)

In [331]:
global_item_sale_mean = transactions.groupby(['item_id','date_block_num'], as_index=False)['item_cnt_day'].sum().groupby(['item_id'])['item_cnt_day'].mean()
transactions['global_item_sale_mean'] = transactions['item_id'].map(global_item_sale_mean)

In [332]:
month_item_sale = transactions.groupby(['item_id','date_block_num'], as_index=False)['item_cnt_day'].sum()
transactions['global_item_sale_mean'] = transactions['item_id'].map(global_item_sale_mean)

In [333]:
month_sums = transactions.groupby(['item_id','date_block_num'],as_index=False)['item_cnt_day'].sum().rename(columns={'item_cnt_day':'item_sale_month'})
transactions = transactions.merge(month_sums, on=['item_id','date_block_num'], how="left")

In [334]:
number_of_days_with_a_sale = transactions.groupby(['item_id', 'date'], as_index=False).first().groupby('item_id').size()

In [335]:
transactions['number_of_days_with_a_sale'] = transactions['item_id'].map(number_of_days_with_a_sale)

In [336]:
number_of_months_with_a_sale = transactions.groupby(['item_id', 'date_block_num'], as_index=False).first().groupby('item_id').size()

In [337]:
transactions['number_of_months_with_a_sale'] = transactions['item_id'].map(number_of_months_with_a_sale)

In [ ]:
transactions['max_day_sale'] = transactions.groupby('item_id')['item_cnt_day'].transform(np.max)
transactions['min_day_sale'] = transactions.groupby('item_id')['item_cnt_day'].transform(np.min)

transactions['max_day_month'] = transactions.groupby(['item_id','date_block_num'])['item_cnt_day'].transform(np.max)
transactions['min_day_sale'] = transactions.groupby('item_id')['item_cnt_day'].transform(np.min)

In [354]:
month_max = transactions.groupby(['item_id','date_block_num'],as_index=False)['item_cnt_day'].sum().groupby(['item_id','date_block_num'],as_index=False)['item_cnt_day'].max()\
        .rename(columns={'item_cnt_day':'item_max_sales_month'})

transactions = transactions.merge(month_max, on=['item_id','date_block_num'], how="left")

In [355]:
month_min = transactions.groupby(['item_id','date_block_num'],as_index=False)['item_cnt_day'].sum().groupby(['item_id','date_block_num'],as_index=False)['item_cnt_day'].min()\
        .rename(columns={'item_cnt_day':'item_min_sales_month'})

transactions = transactions.merge(month_min, on=['item_id','date_block_num'], how="left")

In [338]:
def get_number_of_days_since_start(day,month, year):
    days = 0
    if year == 2015:
        days = 365
    def is_even(num):
        return num % 2 == 0
    half_of_month = int(month/2)
    even = (30*half_of_month) + (31*half_of_month)
    if is_even(month):
        days = days + even - 30 - day
    else:
        days = days + even + day
    return days

In [339]:
transactions['number_of_days_since_beginning'] = transactions.apply(lambda row: get_number_of_days_since_start(row['day'],row['month'], row['year']),axis=1)

In [340]:
gc.collect()

def get_max_stretch_without_sales_days(days):
    days  = [0] + days + [365+365-31-30]
    days = np.unique(days)
    max_stretch = 0
    len_days = len(days)
    for index,day in enumerate(sorted(days)):
        if index == len_days - 1:
            return max_stretch
        next_day = days[index+1]
        stretch = next_day - day
        if stretch > max_stretch:
            max_stretch = stretch
            

        
max_stretch_without_sales = transactions.groupby(['item_id', 'date'], as_index=False).first().groupby(['item_id'])['number_of_days_since_beginning']\
                                    .apply(list).apply(lambda x: get_max_stretch_without_sales_days(x))

transactions['max_stretch_in_days_without_sales'] = transactions['item_id'].map(max_stretch_without_sales)

In [341]:
transactions['date_block_num'].max()

33

In [342]:
gc.collect()

def get_max_stretch_without_sales_block(blocks):
    blocks  = [12] + blocks + [33]
    blocks = np.unique(blocks)
    max_stretch = 0
    len_blocks = len(blocks)
    for index,block in enumerate(sorted(blocks)):
        if index == len_blocks - 1:
            return max_stretch
        next_block = blocks[index+1]
        stretch = next_block - block
        if stretch > max_stretch:
            max_stretch = stretch
            

        
max_stretch_without_sales_block = transactions.groupby(['item_id'])['date_block_num']\
                                    .apply(list).apply(lambda x: get_max_stretch_without_sales_block(x))

transactions['max_stretch_in_blocks_without_sales'] = transactions['item_id'].map(max_stretch_without_sales_block)

In [343]:
def get_following_pairs(days):
    days = np.unique(days)
    len_days = len(days)
    following = []
    for index,day in enumerate(sorted(days)):
        if index == len_days - 1:
            return following
        next_day = days[index+1]
        if next_day == day + 1:
            following.append([day, next_day])
        
assert(get_following_pairs([1,2,5,6,7,8,9,11,12,15]) == [[1, 2], [5, 6], [6, 7], [7, 8], [8, 9], [11, 12]])
assert(get_following_pairs([1,2,5,6,7,10]) == [[1, 2], [5, 6], [6, 7]])
assert(get_following_pairs([1,2,4,5,7,9,10]) == [[1, 2], [4, 5], [9,10]])
assert(get_following_pairs([1,2,4,5,7,9,10,11,12,15]) == [[1, 2], [4, 5], [9,10],[10,11],[11,12]])





def get_longest_stretch(following_pairs, n=1):
    len_pairs = len(following_pairs)
    if len_pairs == 0:
        return 0
    if len_pairs == 1:
        return n
    if following_pairs[1][0] == following_pairs[0][1]:
        n+=1
    return get_longest_stretch(following_pairs[1:], n)


assert(get_longest_stretch([]) == 0)
assert(get_longest_stretch([[1, 2], [2,3], [3, 4], [4,5] ,[8,9], [11, 12]]) == 4)
assert(get_longest_stretch([[1, 2], [4,5], [7, 8]]) == 1)
assert(get_longest_stretch([[1, 2], [5, 6], [6, 7], [7,8], [14, 15]]) == 3)

gc.collect()

max_stretch_with_sales_days = transactions.groupby(['item_id', 'date'], as_index=False).first().groupby(['item_id'])['number_of_days_since_beginning']\
                                    .apply(list).apply(lambda x: get_longest_stretch(get_following_pairs(x)))

transactions['max_stretch_in_days_with_sales'] = transactions['item_id'].map(max_stretch_with_sales_days)

gc.collect()

max_stretch_with_sales_blocks = transactions.groupby(['item_id'])['date_block_num']\
                                    .apply(list).apply(lambda x: get_longest_stretch(get_following_pairs(x)))

transactions['max_stretch_in_blocks_with_sales'] = transactions['item_id'].map(max_stretch_with_sales_blocks)


In [344]:
def get_average_days_between_sales(days):
    days = sorted(np.unique(days))
    if len(days) == 0:
        return 9999
    if len(days) == 1:
        return 999
    return np.mean(np.ediff1d(days)) / len(days)

average_days_between_sales = transactions.groupby(['item_id', 'date'], as_index=False).first().groupby(['item_id'])['number_of_days_since_beginning']\
                                    .apply(list).apply(lambda x: get_average_days_between_sales(x))

transactions['average_days_between_sales'] = transactions['item_id'].map(average_days_between_sales)

In [345]:
transactions['possibly_released_during_period'] = ((transactions['max_stretch_in_blocks_without_sales'] > 8) & (transactions['number_of_days_with_a_sale'] > 200)).astype(bool)

In [346]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
transactions.sample(10).sort_values(by=['total_sales_units'], ascending=False)

,item_id,date,date_block_num,shop_id,item_price,item_cnt_day,day,month,year,item_name,item_category_id,y,total_sales_units,avg_sales_month,global_item_sale_mean,item_sale_month,number_of_days_with_a_sale,number_of_months_with_a_sale,number_of_days_since_beginning,max_stretch_in_days_without_sales,max_stretch_in_blocks_without_sales,max_stretch_in_days_with_sales,max_stretch_in_blocks_with_sales,average_days_between_sales,possibly_released_during_period
256741,3329,19.02.2014,13,58,1249.5,2.0,19,2,2014,"FIFA 14 (с поддержкой PS Move) [PS3, русская версия]",19,13.0,2362.0,410.0,107.363636,410.0,514,22,12,33,1,246,21,0.006950,False
383340,4248,11.06.2014,17,4,1049.0,1.0,11,6,2014,Kinect Sports Ultimate (только для MS Kinect) [Xbox 360],23,5.0,2306.0,134.0,104.818182,134.0,553,22,142,31,1,289,21,0.005975,False
136529,2252,09.01.2015,24,19,399.0,1.0,9,1,2015,"Call Of Duty: Modern Warfare 3 [PC, Jewel]",30,7.0,1989.0,150.0,90.409091,150.0,608,22,374,31,1,313,21,0.005535,False
660761,6710,29.06.2015,29,28,599.0,1.0,29,6,2015,"Spintires [PC, русская версия]",30,3.0,1846.0,57.0,108.588235,57.0,452,17,489,123,5,240,16,0.007589,False
1526128,19985,19.09.2015,32,31,299.0,1.0,19,9,2015,"Трансформеры. Битва за Темную Искру [PC, Jewel, русская версия]",30,4.0,972.0,29.0,60.750000,29.0,360,16,628,183,6,169,15,0.009254,False
679558,7006,19.03.2014,14,21,2999.0,1.0,19,3,2014,"Thief [PS4, русская версия]",20,9.0,891.0,390.0,44.550000,390.0,285,20,80,62,2,125,19,0.011615,False
1449656,18570,03.05.2015,28,22,399.0,1.0,3,5,2015,СЕДЬМОЙ СЫН,40,2.0,389.0,164.0,55.571429,164.0,112,7,490,427,15,58,6,0.034242,False
1570595,20656,14.08.2015,31,38,999.0,1.0,14,8,2015,Фигурка Plants Vs Zombies Conehead Zombie With Sunflower 8см,72,1.0,251.0,6.0,11.409091,6.0,198,22,565,36,1,42,21,0.023147,False
1066795,13241,31.10.2015,33,24,450.0,1.0,31,10,2015,Комикс Ведьмак Лисьи дети,47,3.0,195.0,195.0,195.000000,195.0,25,1,609,609,21,24,0,0.040000,False
1418047,18036,24.06.2015,29,47,299.0,1.0,24,6,2015,Резинки для плетения Розово-голубой перламутр Pearl Pink/Ocean Blue Rainbow Loom арт. BW0016,70,3.0,173.0,68.0,34.600000,68.0,79,5,494,489,17,41,4,0.035817,False
